<a href="https://www.kaggle.com/code/peacehegemony/kaggle-notebook-that-trains-a-model-to-classify?scriptVersionId=119369831" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from PIL import Image
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras import backend as K



def binary_focal_loss(gamma=2.0, alpha=0.25):

    def focal_loss(y_true, y_pred):

        # Define epsilon to avoid divide-by-zero errors
        epsilon = K.epsilon()

        # Add a small value to the predicted values to avoid log(0)
        y_pred = y_pred + epsilon

        # Calculate the focal loss
        focal_loss = -y_true * alpha * K.pow(1 - y_pred, gamma) * K.log(y_pred) \
                     - (1 - y_true) * (1 - alpha) * K.pow(y_pred, gamma) * K.log(1 - y_pred)

        return K.mean(focal_loss)

    return focal_loss


# Use learning rate scheduler to prevent overfitting
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate


# Set path for the dataset
path = '/kaggle/input/494-fox-labeled-image-dataset/images.cv_8rk8h3aklzqjr2tcx76b7/data/train/animal fox'


# Create list to store the images and labels
images = []
labels = []


# Loop through the directory and read in the images
for filename in os.listdir(path):
    if filename.endswith('.jpg'):
        img = Image.open(path + '/' + filename)
        images.append(np.array(img))
        labels.append(1) # Assume all images are of foxes


# Convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)


# Normalize the images
images = images / 255.0


# One-hot encode the labels
labels = to_categorical(labels)


# Get the shape of the first image to use as input_shape for the first layer
img_shape = images[0].shape


# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=img_shape))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))


# Compile the model using the binary focal loss function
model.compile(optimizer='adam', loss=binary_focal_loss(), metrics=['accuracy'])


# Use data augmentation to increase the number of training data
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

datagen.fit(images)


# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


model.history



# Set path for the test dataset
test_path = '/kaggle/input/494-fox-labeled-image-dataset/images.cv_8rk8h3aklzqjr2tcx76b7/data/test/animal fox'


# Create lists to store the test images and labels
test_images = []
test_labels = []


# Loop through the test directory and read in the images
for filename in os.listdir(test_path):
    if filename.endswith('.jpg'):
        img = Image.open(test_path + '/' + filename)
        test_images.append(np.array(img))
        test_labels.append(1) # Assume all images are of foxes


# Convert the test images and labels to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)


# Normalize the test images
test_images = test_images / 255.0


# One-hot encode the test labels
test_labels = to_categorical(test_labels)


# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("")
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

2023-02-16 14:14:40.863682: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-02-16 14:14:41.887465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3/3 [==============================] - 1s 55ms/step - loss: 0.0449 - accuracy: 1.0000

Test loss: 0.04488859698176384
Test accuracy: 1.0


This code is using the model.predict() function to generate predictions for the test images, which are images located in the specified test_path. The model.predict() function returns a probability for each class, in this case, fox or not fox. The predictions variable will contain an array of shape (n, 2) where n is the number of images in the test_path and 2 is the number of classes. Each row of the predictions array will contain the probability for each class. The output of the print function would be the softmax output of the model for each image of the test set.

In [2]:
# Generate a prediction using model.predict() and calculate it's shape:
print("")
print("Generate a prediction")
prediction = model.predict(test_images[:1])
print("Prediction:", prediction)
print("Prediction shape:", prediction.shape)


Generate a prediction
Prediction: [[0.38834074 0.6116592 ]]
Prediction shape: (1, 2)


Note that this is a basic example, and you may need to adjust the code to suit your specific dataset and requirements. Also, the model architecture and parameters used here are simple and may not work well in practice.

# Here is an improved version:

In [3]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from PIL import Image
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras import backend as K



def binary_focal_loss(gamma=2.0, alpha=0.25):

    def focal_loss(y_true, y_pred):

        # Define epsilon to avoid divide-by-zero errors
        epsilon = K.epsilon()

        # Add a small value to the predicted values to avoid log(0)
        y_pred = y_pred + epsilon

        # Calculate the focal loss
        focal_loss = -y_true * alpha * K.pow(1 - y_pred, gamma) * K.log(y_pred) \
                     - (1 - y_true) * (1 - alpha) * K.pow(y_pred, gamma) * K.log(1 - y_pred)

        return K.mean(focal_loss)

    return focal_loss


# Use learning rate scheduler to prevent overfitting
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate


# Set path for the dataset
path = '/kaggle/input/494-fox-labeled-image-dataset/images.cv_8rk8h3aklzqjr2tcx76b7/data/train/animal fox'


# Create list to store the images and labels
images = []
labels = []


# Loop through the directory and read in the images
for filename in os.listdir(path):
    if filename.endswith('.jpg'):
        img = Image.open(path + '/' + filename)
        images.append(np.array(img))
        labels.append(1) # Assume all images are of foxes


# Convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)


# Normalize the images
images = images / 255.0


# One-hot encode the labels
labels = to_categorical(labels)


# Get the shape of the first image to use as input_shape for the first layer
img_shape = images[0].shape


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=img_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


# Compile the model using the binary focal loss function
model.compile(optimizer='adam', loss=binary_focal_loss(), metrics=['accuracy'])


# Use data augmentation to increase the number of training data
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

datagen.fit(images)


# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


model.history



# Set path for the test dataset
test_path = '/kaggle/input/494-fox-labeled-image-dataset/images.cv_8rk8h3aklzqjr2tcx76b7/data/test/animal fox'


# Create lists to store the test images and labels
test_images = []
test_labels = []


# Loop through the test directory and read in the images
for filename in os.listdir(test_path):
    if filename.endswith('.jpg'):
        img = Image.open(test_path + '/' + filename)
        test_images.append(np.array(img))
        test_labels.append(1) # Assume all images are of foxes


# Convert the test images and labels to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)


# Normalize the test images
test_images = test_images / 255.0


# One-hot encode the test labels
test_labels = to_categorical(test_labels)


# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("")
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

3/3 [==============================] - 1s 73ms/step - loss: 0.1320 - accuracy: 0.0130

Test loss: 0.13201497495174408
Test accuracy: 0.012987012974917889


In [4]:
# Generate a prediction using model.predict() and calculate it's shape:
print("")
print("Generate a prediction")
prediction = model.predict(test_images[:1])
print("Prediction:", prediction)
print("Prediction shape:", prediction.shape)


Generate a prediction
Prediction: [[0.51517713 0.48482284]]
Prediction shape: (1, 2)


This improved version of the code:

- Use train_test_split to split the data into train and test sets. This is a good practice to avoid overfitting and generalize the model better.
- Add MaxPooling2D layer to reduce the spatial dimensions of the feature maps, reduce the number of parameters and computation.
- Add Dropout layers to prevent overfitting by randomly disabling some of the neurons in the model during training.
- Add a dense layer with 128 units and relu activation before the output layer
- Also, the input shape of the first layer is passed as parameter.
- This model architecture and parameters are still simple, but this is improved than the previous version and may work better in practice.

In the context of neural network training, an epoch is one full pass through the entire training dataset. During an epoch, the model's parameters are updated based on the error it made in predicting the output for the training examples. The number of epochs is a hyperparameter that determines the number of times the learning algorithm will work through the entire training dataset.

When training a model, it's common to use a large number of epochs (e.g. 50, 100, or even more) to ensure that the model has seen the entire training dataset multiple times, and has had ample opportunity to learn the patterns in the data. However, as the model continues to train, it will eventually reach a point where additional epochs will not significantly improve the model's performance. This is known as overfitting, where the model becomes too specialized to the training data and perform badly on new unseen data.

That's why it's important to monitor the performance of the model on a validation set during training, and stop training once the performance on the validation set stops improving or starts to degrade.

In the example provided, 10 epochs are used. This is a relatively small number of epochs and the model may not have enough time to converge, it's a good practice to increase the number of epochs and check the performance of the model by monitoring the accuracy and loss during the training.